# Ocean Heat Contant

$OHC = \Delta T \times c_p \times \rho \times V$

unit conversion to SI
- $c_p$: erg/g/K = 1e-7J / 1e-3kg / K = 1e-4 J/kg/K $\rightarrow$ 3996 J/kg/K
- $\rho$: g/cm^3 = 1e3 kg/m^3

In [ ]:
# import pickle
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
# plt.style.use('dark_background')
# sns.set()
# sns.set_palette("tab10")
# sns.set_context("notebook", font_scale=1)

In [ ]:
from paths import file_ex_ohc_hires, file_ex_ocn_ctrl, file_ex_ocn_rcp

In [ ]:
from OHC import OHC_integrals
from paths import CESM_filename, path_samoc
from constants import cp_sw
from timeseries import IterateOutputCESM
from xr_DataArrays import create_xr_DataArray, xr_DZ,xr_AREA, xr_HTN, xr_LATS
from xr_integrate import  xr_vol_int

In [ ]:
from regions import boolean_mask

In [ ]:
from xr_integrate import xr_int_zonal, xr_int_zonal_level

# calculating OHC arrays

In [ ]:
%%time
# 1:03-1:20 for 1 year
OHC = OHC_integrals(domain='ocn', run='ctrl', mask_nr=0)
OHC

In [ ]:
%%time
# 1:03-1:20 for 1 year
OHC = OHC_integrals(domain='ocn', run='ctrl', mask_nr=0)
OHC

In [ ]:
OHC.OHC_zonal.sum()#*111000

# Analysis

In [ ]:
OHC_ctrl = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_ctrl.nc', decode_times=False)
OHC_rcp  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_rcp.nc' , decode_times=False)

In [ ]:
# global integrals
plt.plot(OHC_ctrl.OHC_global/1e21, label='CTRL')
plt.plot(OHC_rcp.OHC_global /1e21, label='RCP')
plt.legend()
plt.ylim((2.49e2, 2.53e2))

In [ ]:
# zonal integrals
plt.figure()
OHC_ctrl.OHC_global_levels.T.plot(vmax=2e22)

plt.figure()
OHC_rcp.OHC_global_levels.T.plot()

In [ ]:
# lat-depth section
plt.figure()
OHC_ctrl.OHC_zonal_levels[0,:,:].T.plot()
# plt.colorbar()

plt.figure()
OHC_rcp.OHC_zonal_levels[0,:,:].T.plot()
# plt.colorbar()


In [ ]:
from xr_regression import xr_linear_trends_2D

In [ ]:
# trends
OHC_zonal_levels_trend_ctrl = xr_linear_trends_2D(OHC_ctrl.OHC_zonal_levels, ('lat', 'depth_t'))
OHC_zonal_levels_trend_rcp  = xr_linear_trends_2D(OHC_rcp.OHC_zonal_levels , ('lat', 'depth_t'))

In [ ]:
OHC_zonal_levels_trend_ctrl

In [ ]:
np.shape(X)

In [ ]:
plt.figure(figsize=(8,5))
X, Y = np.meshgrid(OHC_zonal_levels_trend_ctrl.lat, OHC_zonal_levels_trend_ctrl.depth_t)
plt.pcolormesh(X,Y,  OHC_zonal_levels_trend_ctrl.values)
# OHC_zonal_levels_trend_ctrl.plot()
# plt.figure()
# OHC_zonal_levels_trend_rcp.plot()

In [ ]:
OHC_rcp_vert_trend = xr_linear_trends_2D(OHC_rcp.OHC_vertical, ('lat', 'lon'))

In [ ]:
OHC_rcp_vert_trend.plot()